# Обнаружение статистически значимых отличий в уровнях экспрессии генов больных раком
#### В этом задании вы:
* вспомните, что такое t-критерий Стьюдента и для чего он применяется
* сможете применить технику множественной проверки гипотез и увидеть собственными глазами, как она работает на реальных данных
* почувствуете разницу в результатах применения различных методов поправки на множественную проверку

## Описание используемых данных
Данные для этой задачи взяты из исследования, проведенного в Stanford School of Medicine. В исследовании была предпринята попытка выявить набор  генов, которые позволили  бы более точно диагностировать возникновение рака груди на самых ранних стадиях.

В эксперименте принимали участие 24 человек, у которых не было рака груди (normal), 25 человек, у которых это заболевание было диагностировано на ранней стадии (early neoplasia), и 23 человека с сильно выраженными симптомами (cancer).

Ученые провели секвенирование биологического материала испытуемых, чтобы понять, какие из этих генов наиболее активны в клетках больных людей. 

Секвенирование — это определение степени активности генов в анализируемом образце с помощью подсчёта количества соответствующей каждому гену РНК.

В данных для этого задания вы найдете именно эту количественную меру активности каждого из 15748 генов у каждого из 72 человек, принимавших участие в эксперименте.

Вам нужно будет определить те гены, активность которых у людей в разных стадиях заболевания отличается статистически значимо.

Кроме того, вам нужно будет оценить не только статистическую, но и практическую значимость этих результатов, которая часто используется в подобных исследованиях.

Диагноз человека содержится в столбце под названием "Diagnosis".

## Практическая значимость изменения
Цель исследований — найти гены, средняя экспрессия которых отличается не только статистически значимо, но и достаточно сильно. В экспрессионных исследованиях для этого часто используется метрика, которая называется fold change (кратность изменения). 

Определяется она следующим образом:
\begin{equation*}
F_c(C, T) = 
 \begin{cases}
   \frac{T}{C}, &\text{$T>C$}\\
   -\frac{C}{T}, &\text{$T<C$}
 \end{cases}
\end{equation*}


где  $C, T$ — средние значения экспрессии гена в control и treatment группах соответственно. По сути, fold change показывает, во сколько раз отличаются средние двух выборок.

### Инструкции к решению задачи
Задание состоит из трёх частей. Если не сказано обратное, то уровень значимости нужно принять равным ***0.05***.

#### Загрузка данных:

In [1]:
import pandas as pd
import numpy as np
import scipy
import statsmodels.stats.multitest as smm
from statsmodels.stats.weightstats import *

data = pd.read_csv("gene_high_throughput_sequencing.csv", header=0)
data.head()

,Patient_id,Diagnosis,LOC643837,LOC100130417,SAMD11,NOC2L,KLHL17,PLEKHN1,C1orf170,HES4,...,CLIC2,RPS4Y1,ZFY,PRKY,USP9Y,DDX3Y,CD24,CYorf15B,KDM5D,EIF1AY
0,STT5425_Breast_001_normal,normal,1.257614,2.408148,13.368622,9.494779,20.880435,12.722017,9.494779,54.349694,...,4.761250,1.257614,1.257614,1.257614,1.257614,1.257614,23.268694,1.257614,1.257614,1.257614
1,STT5427_Breast_023_normal,normal,4.567931,16.602734,42.477752,25.562376,23.221137,11.622386,14.330573,72.445474,...,6.871902,1.815112,1.815112,1.815112,1.815112,1.815112,10.427023,1.815112,1.815112,1.815112
2,STT5430_Breast_002_normal,normal,2.077597,3.978294,12.863214,13.728915,14.543176,14.141907,6.232790,57.011005,...,7.096343,2.077597,2.077597,2.077597,2.077597,2.077597,22.344226,2.077597,2.077597,2.077597
3,STT5439_Breast_003_normal,normal,2.066576,8.520713,14.466035,7.823932,8.520713,2.066576,10.870009,53.292034,...,5.200770,2.066576,2.066576,2.066576,2.066576,2.066576,49.295538,2.066576,2.066576,2.066576
4,STT5441_Breast_004_normal,normal,2.613616,3.434965,12.682222,10.543189,26.688686,12.484822,1.364917,67.140393,...,11.227770,1.364917,1.364917,1.364917,1.364917,1.364917,23.627911,1.364917,1.364917,1.364917


### Часть 1: применение t-критерия Стьюдента
В первой части вам нужно будет применить критерий Стьюдента для проверки гипотезы о равенстве средних в двух независимых выборках. Применить критерий для каждого гена нужно будет дважды:

   1) для групп **normal (control)** и **early neoplasia (treatment)**  
   2) для групп **early neoplasia (control)** и **cancer (treatment)**

В качестве ответа в этой части задания необходимо указать количество статистически значимых отличий, которые вы нашли с помощью t-критерия Стьюдента, то есть число генов, у которых p-value этого теста оказался меньше, чем уровень значимости. 

In [2]:
normal = data.loc[data.Diagnosis == 'normal', data.columns[2:]].reset_index(drop=True)
early = data.loc[data.Diagnosis == 'early neoplasia', data.columns[2:]].reset_index(drop=True)
cancer = data.loc[data.Diagnosis == 'cancer', data.columns[2:]].reset_index(drop=True)
data_columns = data.columns[2:]

В первую очередь проверим признаки на нормальность критерием Шапиро-Уилка, и сразу делаем поправку методом Бенджамини-Хохберга, принимая признаки как независимые:

In [3]:
shap_w = pd.DataFrame() 
for i, col_name in enumerate(data_columns):
    shap_w.loc[i, 'normal'] = stats.shapiro(normal[col_name])[1]
    shap_w.loc[i, 'early neoplasia'] = stats.shapiro(early[col_name])[1]
    shap_w.loc[i, 'cancer'] = stats.shapiro(cancer[col_name])[1]

In [4]:
shap_w['normal_corrected'] = smm.multipletests(shap_w['normal'], method="fdr_bh")[1]
shap_w['early_corrected'] = smm.multipletests(shap_w['early neoplasia'], method="fdr_bh")[1]
shap_w['cancer_corrected'] = smm.multipletests(shap_w['cancer'], method="fdr_bh")[1]

In [5]:
(shap_w<0.05).sum()

normal              4134
early neoplasia     3933
cancer              4253
normal_corrected    2306
early_corrected     2086
cancer_corrected    2334
dtype: int64

Итак, примерно 6/7 всех признаков распределены нормально.

Важно вспомнить, что критерий Стьюдента для выборок с неизвестными дисперсиями также не всегда работает, только при условии:
* $n_1 = n_2$, или
* $[n_1 > n_2] = [\sigma_1 > \sigma_2]$

В нашем случае все три выборки приблизитильно одинаковы по размеру.
Теперь можно использовать критерий Стьюдента.

In [6]:
def compare_genes_diffs(right, left, columns):
    return [scipy.stats.ttest_ind(right[col_name], left[col_name], equal_var = False).pvalue for col_name in columns]

ttest = pd.DataFrame()
ttest['normal=early'] = compare_genes_diffs(normal, early, data_columns)
ttest['early=cancer'] = compare_genes_diffs(early, cancer, data_columns)

print('Т-критерий Стьюдента для отличия распределений генов групп normal и early neoplasia: ', (ttest<0.05).sum()[0])
print('Т-критерий Стьюдента для отличия распределений генов групп early neoplasia и cancer: ', (ttest<0.05).sum()[1])

Т-критерий Стьюдента для отличия распределений генов групп normal и early neoplasia:  1575
Т-критерий Стьюдента для отличия распределений генов групп early neoplasia и cancer:  3490


### Часть 2: поправка методом Холма
Для этой части задания вам понадобится модуль **multitest** из **statsmodels**.

**import** *statsmodels.stats.multitest **as** *smm*

В этой части задания нужно будет применить поправку Холма для получившихся двух наборов достигаемых уровней значимости из предыдущей части. Обратите внимание, что поскольку вы будете делать поправку для каждого из двух наборов **p-value** отдельно, то проблема, связанная с множественной проверкой останется.

Для того, чтобы ее устранить, достаточно воспользоваться поправкой Бонферрони, то есть использовать уровень значимости ***0.05 / 2*** вместо ***0.05*** для дальнейшего уточнения значений **p-value** c помощью метода Холма.

В качестве ответа к этому заданию требуется ввести количество значимых отличий в каждой группе после того, как произведена коррекция Холма-Бонферрони. Причем это число нужно ввести с учетом практической значимости: посчитайте для каждого значимого изменения fold change и выпишите в ответ число таких значимых изменений, абсолютное значение fold change которых больше, чем ***1.5***

Обратите внимание, что:

* применять поправку на множественную проверку  нужно  **ко всем  значениям достигаемых уровней значимости, а не только для тех, которые меньше значения уровня доверия.**  
* при использовании поправки на уровне значимости ***0.025*** **меняются значения достигаемого уровня значимости, но не меняется значение уровня доверия** (то есть для отбора значимых изменений скорректированные значения уровня значимости нужно сравнивать с порогом ***0.025***, а не ***0.05***)!

Количество статистически значимых генов с поправкой Холма:

In [7]:
ttest['n-e_rejects_holm'] = smm.multipletests(ttest['normal=early'], alpha=0.025, method="holm")[0]
ttest['e-c_rejects_holm'] = smm.multipletests(ttest['early=cancer'], alpha=0.025, method="holm")[0]
print('normal и early neoplasia: ', ttest.sum()[2])
print ('early neoplasia и cancer: ', ttest.sum()[3])

normal и early neoplasia:  2.0
early neoplasia и cancer:  79.0


Количество практически значимых генов:

In [8]:
def Fold_change(C, T, column):
    C_mean = C[column].mean()
    T_mean = T[column].mean()
    return abs(T_mean/C_mean if T_mean > C_mean else -C_mean/T_mean)

Fold_changes = pd.DataFrame()
Fold_changes['normal_early'] = [Fold_change(normal, early, col_name) for col_name in data_columns]
Fold_changes['early_cancer'] = [Fold_change(early, cancer, col_name) for col_name in data_columns]
(Fold_changes > 1.5).sum()

normal_early     261
early_cancer    1071
dtype: int64

Количество статистически и практически значимых генов с поправкой Холма:

In [9]:
n_normal_early = len(Fold_changes.loc[(Fold_changes.normal_early>1.5) & (ttest['n-e_rejects_holm']), 'normal_early'])
n_early_cancer = len(Fold_changes.loc[(Fold_changes.early_cancer>1.5) & (ttest['e-c_rejects_holm']), 'early_cancer'])
print('Количество генов со значимым отличием средней экспрессии между группами')
print('*   normal и early neoplasia: ', n_normal_early)
print('*   early neoplasia и cancer: ', n_early_cancer)

Количество генов со значимым отличием средней экспрессии между группами
*   normal и early neoplasia:  2
*   early neoplasia и cancer:  77


### Часть 3: поправка методом Бенджамини-Хохберга
Данная часть задания аналогична второй части за исключением того, что нужно будет использовать метод Бенджамини-Хохберга.

 Обратите внимание, что методы коррекции, которые контролируют **FDR**, допускает больше ошибок первого рода и имеют большую мощность, чем методы, контролирующие **FWER**. Большая мощность означает, что эти методы будут совершать меньше ошибок второго рода (то есть будут лучше улавливать отклонения от $H_0$, когда они есть, и будут чаще отклонять $H_0$, когда отличий нет).

В качестве ответа к этому заданию требуется  ввести количество значимых отличий в каждой группе после того, как произведена коррекция Бенджамини-Хохберга, причем так же, как и во второй части, считать только такие отличия, у которых ***abs(fold change) > 1.5***. 

**P.S.** Вспомните, какое значение имеет уровень значимости $\alpha$ в каждой из поправок:  Холма и Бенджамини-Хохберга. Одинаковый ли смысл имеет уровень значимости в каждой из поправок?

Количество статистически значимых генов c поправкой Бенджамини-Хохберга:

In [10]:
ttest['n-e_rejects_fdr_bh'] = smm.multipletests(ttest['normal=early'], alpha=0.025, method="fdr_bh")[0]
ttest['e-c_rejects_fdr_bh'] = smm.multipletests(ttest['early=cancer'], alpha=0.025, method="fdr_bh")[0]
print('normal и early neoplasia:: ', ttest.sum()[4])
print ('early neoplasia и cancer: ', ttest.sum()[5])

normal и early neoplasia::  4.0
early neoplasia и cancer:  832.0


Количество статистически и практически значимых генов c поправкой Бенджамини-Хохберга:

In [11]:
n_normal_early = len(Fold_changes.loc[(Fold_changes.normal_early>1.5) & (ttest['n-e_rejects_fdr_bh']), 'normal_early'])
n_early_cancer = len(Fold_changes.loc[(Fold_changes.early_cancer>1.5) & (ttest['e-c_rejects_fdr_bh']), 'early_cancer'])
print('Количество генов со значимым отличием средней экспрессии между группами')
print('*   normal и early neoplasia: ', n_normal_early)
print('*   early neoplasia и cancer: ', n_early_cancer)

Количество генов со значимым отличием средней экспрессии между группами
*   normal и early neoplasia:  4
*   early neoplasia и cancer:  524
